In [31]:
import pandas as pd
import numpy as np
import dask as ds

In [32]:
data = pd.read_table("../data/drugs.tsv", sep='\t')
data = data.set_index('CASEID')
data

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,99,99,19,2012,7,1,1,99,...,1,4,1,1,2,2,2,4398.40,30017,1
2,88530883,1,99,99,14,9999,99,2,93,93,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,99,99,14,9999,99,1,2,99,...,1,99,9,9,3,3,2,14052.62,30028,1
4,37814127,1,99,99,16,9999,99,4,93,93,...,1,4,1,1,1,1,2,10848.18,30055,2
5,18762590,1,99,99,14,9999,99,4,93,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55156,13675473,2,99,99,991,9991,91,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55157,49609908,1,99,99,16,9999,99,4,93,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55158,81795924,2,99,99,991,9991,91,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1


In [33]:
# we only want ages 12 - 25
data = data[(data['CATAGE'] == 1) | (data['CATAGE'] == 2)]
data

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,99,99,19,2012,7,1,1,99,...,1,4,1,1,2,2,2,4398.40,30017,1
2,88530883,1,99,99,14,9999,99,2,93,93,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,99,99,14,9999,99,1,2,99,...,1,99,9,9,3,3,2,14052.62,30028,1
8,35106150,1,99,99,22,9999,99,2,93,93,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,99,99,16,9999,99,1,20,99,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,99,99,991,9991,91,91,91,91,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,99,99,18,9999,99,3,93,93,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,99,99,991,9991,91,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1


In [34]:
data['QUESTID2'].value_counts().sum() # length is the same, each Questid is unique
data['CIGOFRSM'].value_counts() # we can see there is majority NaN

99    21177
4     12512
3      1998
2       166
1        13
97        6
94        5
98        1
Name: CIGOFRSM, dtype: int64

We should also check how many columns are there where the data is recorded as floats instead of integers, which can alter our missingness

In [35]:
float_cols_list = data.select_dtypes(include=['float64']).columns.tolist()
float_cols_list

['IRCIGFM',
 'IRCGRFM',
 'IRCHWFM',
 'IRSNFFM',
 'IRALCFM',
 'IRMJFM',
 'IRCOCFM',
 'IRHERFM',
 'IRHALFM',
 'IRINHFM',
 'CIGAVGD',
 'CIGAVGM',
 'ALCAVGM',
 'COLDYR1',
 'COLDYR2',
 'COLDYR3',
 'COLDYR4',
 'IRCGIRTB',
 'IRCGCRV',
 'IRCGCRGP',
 'IRCGNCTL',
 'IRCGAVD',
 'IRCGPLN',
 'IRCGROUT',
 'IRCGRGWK',
 'IRCGRGNM',
 'IRCGNCG',
 'IRCGSLHR',
 'IRCGINFL',
 'IRCGNINF',
 'IRCGINCR',
 'IRCGLMR',
 'NDSSANSP',
 'BMI2',
 'SMIPP_U',
 'ANALWT_C']

In [36]:
# create a subset of the df to see what these columns look like
floats = data[float_cols_list]
floats

,IRCIGFM,IRCGRFM,IRCHWFM,IRSNFFM,IRALCFM,IRMJFM,IRCOCFM,IRHERFM,IRHALFM,IRINHFM,...,IRCGNCG,IRCGSLHR,IRCGINFL,IRCGNINF,IRCGINCR,IRCGLMR,NDSSANSP,BMI2,SMIPP_U,ANALWT_C
CASEID,,,,,,,,,,,,,,,,,,,,,
1,1.0,93.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,...,4.0,2.0,5.0,1.0,2.0,1.0,1.35,23.41,0.006,4398.40
2,93.0,93.0,91.0,15.0,2.0,30.0,1.0,91.0,1.0,91.0,...,93.0,93.0,93.0,93.0,93.0,93.0,-9.00,21.77,0.007,1419.19
3,2.0,91.0,93.0,91.0,93.0,91.0,91.0,91.0,91.0,91.0,...,1.0,1.0,5.0,1.0,1.0,2.0,1.82,18.65,-9.000,14052.62
8,93.0,91.0,91.0,91.0,10.0,91.0,91.0,91.0,91.0,91.0,...,93.0,93.0,93.0,93.0,93.0,93.0,-9.00,24.39,0.011,3211.24
9,20.0,93.0,93.0,30.0,18.0,93.0,93.0,91.0,93.0,91.0,...,2.0,1.0,4.0,2.0,1.0,2.0,1.71,26.30,0.011,6396.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,...,91.0,91.0,91.0,91.0,91.0,91.0,-9.00,-9.00,0.003,1649.35
55155,93.0,93.0,93.0,91.0,17.0,93.0,91.0,91.0,93.0,91.0,...,93.0,93.0,93.0,93.0,93.0,93.0,-9.00,26.15,0.004,4594.84
55156,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,91.0,...,91.0,91.0,91.0,91.0,91.0,91.0,-9.00,33.12,0.074,679.36


In [37]:
def check_missing(x):
    """
    Checks whether or not a value in a series ends with a certain number

    Parameters
    ----------
    x : int
        value from series
    
    Returns
    -------
    bool : True if it does end with the number otherwise false
    """
    str_x = str(x)
    return str_x.endswith(('91', '93', '94', '97', '98', '99', '.0'))

In [38]:
# drop columns where majority are NaN
data = data.loc[:, data.applymap(check_missing).mean() <= .9]
data

,QUESTID2,CIGEVER,SNFEVER,CHEWEVER,CIGAREVR,PIPEVER,ALCEVER,ALCTRY,ALCREC,ALCYRTOT,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,48694667,1,2,2,1,2,2,991,91,991,...,1,4,1,1,2,2,2,4398.40,30017,1
2,88530883,1,1,2,1,2,1,13,1,30,...,1,4,1,1,1,1,2,1419.19,30052,2
3,33251077,1,2,1,2,2,1,16,3,993,...,1,99,9,9,3,3,2,14052.62,30028,1
8,35106150,1,2,2,2,1,1,19,1,104,...,1,1,1,1,2,2,2,3211.24,30024,1
9,67182690,1,1,1,1,1,1,14,1,104,...,1,4,1,1,1,1,2,6396.73,30038,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55153,31776420,2,2,2,2,2,2,991,91,991,...,1,4,1,1,2,2,2,1649.35,30024,2
55155,10772244,1,2,1,1,1,1,15,1,208,...,1,1,1,1,1,1,2,4594.84,30002,2
55156,13675473,2,2,2,2,2,2,991,91,991,...,1,2,1,1,3,3,2,679.36,30003,1


In [39]:
float_cols_list = data.select_dtypes(include=['float64']).columns.tolist()
float_cols_list # now we can see the list is much smaller

['BMI2', 'SMIPP_U', 'ANALWT_C']